In [ ]:
%pip install pedalboard

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [22]:
from pedalboard import load_plugin

proc2_path = './plugins/FabFilter Pro-C 2.vst3'
prog_path = './plugins/FabFilter Pro-G.vst3'

def config_c2(numOfLevels, currentLevel, c2):
  c2.knee = (72 / numOfLevels) * currentLevel
  c2.range =  1
  c2.auto_release = True
  c2.release = 100
  c2.attack = 33

proC2 = load_plugin(proc2_path)

In [23]:
def config_prog(prog): 
  prog.threshold = -45
  prog.attack = 0
  prog.hold = 77
  prog.knee = 8
  prog.release = 1.0
  prog.lookahead_enabled = True
  prog.lookahead = 1
  prog.left_side_chain_mix = -1
  prog.right_side_chain_mix = 1
  return prog

  
prog = config_prog(load_plugin(prog_path))

In [24]:
import numpy as np

def normalize(audio):
    max_val = np.max(np.abs(audio))
    if max_val > 0:
        audio = audio / max_val
    return audio

def bounce(audios):
  bounce = audios[0]
  for i in range (1, len(audios)):
    bounce = bounce + audios[i]
  return normalize(bounce)

In [25]:
depth_config = [
  ['TALENTHOUSE-DSMN-LV-STEM.flac'],
  ['TALENTHOUSE-DSMN-BV STEM.flac'],
  ['TALENTHOUSE-DSMN-PIANO STEM.flac'], 
  ['TALENTHOUSE-DSMN-GTR STEM.flac'],
  ['TALENTHOUSE-DSMN-KICK STEM.flac'],
  ['TALENTHOUSE-DSMN-KIT STEM.flac'],
  ['TALENTHOUSE-DSMN-PERC STEM.flac'], 
  ['TALENTHOUSE-DSMN-BASS STEM.flac'],
  ['TALENTHOUSE-DSMN-SNARE STEM.flac'],
  ['TALENTHOUSE-DSMN-LV-STEM.flac'],
  # ['LEAD VOX - U48 - Neve 33102.wav'],
  # ['UPRIGHT BASS - DI - Neve 33102.wav', 'UPRIGHT BASS - ELA M 260 - Neve 33102.wav'],
  # ['TROMBONE A - AR-51 - Neve 33102.wav', 'TROMBONE B - AR-51 - Neve 33102.wav', 'TRUMPET A - AR-51 - Neve 33102.wav', 'TRUMPET B - AR-51 - Neve 33102.wav', 'SAX A - AK-47mkII - Neve 33102.wav', 'SAX B - AK-47mkII - Neve 33102.wav'],
  # ['DRUM OVERHEAD STEREO L - AR-70 - Neve 33102.wav', 'DRUM OVERHEAD STEREO R - AR-70 - Neve 33102.wav', 'KICK - U48 - Neve 33102.wav'],
  # ['E GUITAR - AR-51 - Presonus M80.wav', 'PIANO A - ELA M 260 - Neve 33102.wav', 'PIANO B - ELA M 260 - Neve 33102.wav'],
  # ['ROOM A - Beyer Dynamic 160 - Neve 33102.wav', 'ROOM B - Beyer Dynamic 160 - Neve 33102.wav', 'ROOM C - AR-51 - Chandler Germanium.wav', 'ROOM D - AR-51 - Chandler Germanium.wav', 'ROOM STEREO L - C24 w TFUNK CK12 - Neve 33102.wav', 'ROOM STEREO R - C24 w TFUNK CK12 - Neve 33102.wav']
]

In [26]:
from pedalboard.io import AudioFile
from pedalboard import Pedalboard
import numpy as np
import os

def mix_files(depth_config):
    outputs = []
    inputs = []
    levels = len(depth_config)
    max_samples = 0
    samplerate = 44100
    for i in range(0, levels):
      songs_at_depth = depth_config[i]
      for j in range(0, len(songs_at_depth)):
        file_path = './uploads/' + songs_at_depth[j]
        print('Processing ' + songs_at_depth[j])
        f = AudioFile(file_path)
        audio = f.read(f.frames)
        samplerate = f.samplerate
        max_samples = max(max_samples, audio.shape[1])
        inputs.append(audio)
        pc = proC2;
        config_c2(len(depth_config), i, pc)
        pg = prog;
        board = Pedalboard([pg, pc])
        outputs.append(board(audio, samplerate, reset=False))
        print('Processed ' + songs_at_depth[j])
    for i in range(len(inputs)):
      shape = inputs[i].shape
      if shape[1] < max_samples:
        if (shape[0] == 1):
          inputs[i] = np.pad(inputs[i], (0, max_samples - shape[1]), 'constant')
          outputs[i] = np.pad(outputs[i], (0, max_samples - shape[1]), 'constant')
        else:
          inputs[i] = np.pad(inputs[i], ((0,0), (0, max_samples - shape[1])), 'constant')
          outputs[i] = np.pad(outputs[i], ((0,0), (0, max_samples - shape[1])), 'constant')

    # Normalize the mixed audio to prevent clipping
    mixed_audio = bounce(outputs)
    pre_mixed_audio = bounce(inputs)

    # Save the output
    output_file = 'mixdown/mixed_output.wav'
    output_file2 = 'mixdown/pre_mixed_output.wav'
    if not os.path.exists('mixdown'):
      os.makedirs('mixdown')
    with AudioFile(output_file, 'w', samplerate, outputs[0].shape[0]) as f:
        f.write(mixed_audio)
    with AudioFile(output_file2, 'w', samplerate, outputs[0].shape[0]) as f:
        f.write(pre_mixed_audio)
    return [output_file, output_file2]

In [27]:
mix_files(depth_config)

Processing TALENTHOUSE-DSMN-LV-STEM.flac
Processed TALENTHOUSE-DSMN-LV-STEM.flac
Processing TALENTHOUSE-DSMN-BV STEM.flac
Processed TALENTHOUSE-DSMN-BV STEM.flac
Processing TALENTHOUSE-DSMN-PIANO STEM.flac
Processed TALENTHOUSE-DSMN-PIANO STEM.flac
Processing TALENTHOUSE-DSMN-GTR STEM.flac
Processed TALENTHOUSE-DSMN-GTR STEM.flac
Processing TALENTHOUSE-DSMN-KICK STEM.flac
Processed TALENTHOUSE-DSMN-KICK STEM.flac
Processing TALENTHOUSE-DSMN-KIT STEM.flac
Processed TALENTHOUSE-DSMN-KIT STEM.flac
Processing TALENTHOUSE-DSMN-PERC STEM.flac
Processed TALENTHOUSE-DSMN-PERC STEM.flac
Processing TALENTHOUSE-DSMN-BASS STEM.flac
Processed TALENTHOUSE-DSMN-BASS STEM.flac
Processing TALENTHOUSE-DSMN-SNARE STEM.flac
Processed TALENTHOUSE-DSMN-SNARE STEM.flac
Processing TALENTHOUSE-DSMN-LV-STEM.flac
Processed TALENTHOUSE-DSMN-LV-STEM.flac


['mixdown/mixed_output.wav', 'mixdown/pre_mixed_output.wav']